In [ ]:
from torchvision.models.segmentation import deeplabv3
import torch.nn as nn
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from utils import data
import pandas as pd
from torch.utils.data import Subset, DataLoader
from torchvision import transforms
import config
from tqdm import tqdm
import kornia
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Should be cuda:0 in colab and cpu in local.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
DATA_PATH = config.PATH_TO_DATA

train_test_split = pd.read_csv("dataset/data_split.csv")
train_split = train_test_split[train_test_split["split"] == "Train"]["sampleid"].values
test_split = train_test_split[train_test_split["split"] == "Test"]["sampleid"].values

In [ ]:
preprocess = transforms.Compose(
    [
        transforms.Resize(
            (256, 256)
        ),  # Optional: Resize the input PIL Image to the given size.
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)
preprocess_mask = transforms.Compose(
    [
        kornia.geometry.Resize(
            (256, 256), interpolation="nearest"
        ),  # Optional: Resize the input PIL Image to the given size.
    ]
)
dataset = data.PlanetDataset(
    data_dir=DATA_PATH,
    bands=[0, 1, 2],
    transform=preprocess,
    target_transform=preprocess_mask,
)

In [ ]:
print((dataset.id2index))

training_set = Subset(
    dataset=dataset, indices=[dataset.id2index[image_id] for image_id in train_split]
)
test_set = Subset(
    dataset=dataset, indices=[dataset.id2index[image_id] for image_id in test_split]
)

In [ ]:
print(
    f"The training split has {len(training_set)} samples ({len(training_set)/len(dataset):.2%} of the full dataset)."
)

In [ ]:
from pprint import pprint

training_set[0][1].shape

In [ ]:
def prepare_model(backbone_model="mbv3", num_classes=2):
    """Source : https://learnopencv.com/deep-learning-based-document-segmentation-using-semantic-segmentation-deeplabv3-on-custom-dataset/"""
    weights = "DEFAULT"  # Initialize model with pre-trained weights.
    if backbone_model == "mbv3":
        model = deeplabv3.deeplabv3_mobilenet_v3_large(weights=weights)
    elif backbone_model == "r50":
        model = deeplabv3.deeplabv3_resnet50(weights=weights)
    elif backbone_model == "r101":
        model = deeplabv3.deeplabv3_resnet101(weights=weights)
    else:
        raise ValueError(
            "Wrong backbone model passed. Must be one of 'mbv3', 'r50' and 'r101' "
        )

    model.classifier[4] = nn.LazyConv2d(num_classes, 1)
    model.aux_classifier[4] = nn.LazyConv2d(num_classes, 1)

    return model


model = prepare_model(num_classes=2)


def weighted_focal_loss(p, y_true, alpha=0.7, gamma=1):
    """Weighted Focal Loss as described in https://www.mdpi.com/2072-4292/14/19/4694#B26-remotesensing-14-04694"""
    p = p["out"].softmax(dim=1)[
        :, 0, :, :
    ]  # transform model output to probabilities. Following the paper, we consider only the probability of the POSITIVE class
    loss = torch.mean(
        y_true * (-alpha * (torch.ones(size=p.size()) - p) ** gamma * torch.log(p))
        - (torch.ones(size=p.size()) - y_true)
        * (1 - alpha)
        * (p) ** gamma
        * torch.log(p)
    )
    return loss

In [ ]:
max_epochs = 1
params = {"batch_size": 4, "shuffle": True}
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
dataloader = DataLoader(training_set, **params)
criterion = weighted_focal_loss
# Loop over epochs
loss_list = [0]
scheduler = CosineAnnealingLR(optimizer, T_max=len(dataloader) * max_epochs)
for epoch in range(max_epochs):
    # Training
    loss = 0
    progress_bar = tqdm(total=len(dataloader), desc=f"Loss: {loss:.5f}")

    for local_batch, local_labels in dataloader:
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        outputs = model(local_batch)
        loss = criterion(outputs, local_labels)  # Calculate the loss
        loss_list.append(loss.item())
        # Backward pass and optimization
        optimizer.zero_grad()  # Clear gradients
        loss.backward()  # Compute gradients
        optimizer.step()  # Update weights
        scheduler.step()  # Update learning rate
        progress_bar.set_description(f"Loss: {loss.item():.5f}")
        progress_bar.update(1)  # Manually update the progress bar

In [ ]:
fig, ax = plt.subplots()
ax.plot(loss_list)
ax.set_xlabel("Iter")
ax.set_ylabel("Loss")

In [ ]:
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

test_dataloader = DataLoader(test_set, **params)
# tqdm.close()

# Assuming you have a trained model, test data, and labels
model.eval()  # Set the model to evaluation mode
y_true = []  # True labels
y_pred = []  # Predicted labels

with torch.no_grad():
    for local_batch, local_labels in tqdm(test_dataloader):
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        outputs = model(local_batch)["out"]
        _, predicted = torch.max(outputs, 1)

        y_true.extend(local_labels.cpu().numpy())  # Collect true labels
        y_pred.extend(predicted.cpu().numpy())  # Collect predicted labels

# Calculate precision, recall, and F1 score
y_pred = np.array(y_pred).flatten()
y_true = np.array(y_true).flatten()
precision = precision_score(y_true, y_pred, average="weighted")
recall = recall_score(y_true, y_pred, average="weighted")
f1 = f1_score(y_true, y_pred, average="weighted")

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

## DeeplabV3 plus

In [1]:
from models.deeplabv3plus import fetch_deeplabv3
from utils.data import create_dataloaders
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.nn import Softmax

import config

DATA_PATH = config.PATH_TO_DATA

In [2]:
train_loader, val_loader, test_loader = create_dataloaders(
    DATA_PATH, 4, bands=[0, 1, 2]
)
model = fetch_deeplabv3()

In [3]:
def weighted_focal_loss(p, y_true, alpha=0.7, gamma=1):
    """Weighted Focal Loss as described in https://www.mdpi.com/2072-4292/14/19/4694#B26-remotesensing-14-04694"""
    p = p.softmax(dim=1)[
        :, 0, :, :
    ]  # transform model output to probabilities. Following the paper, we consider only the probability of the POSITIVE class
    loss = torch.mean(
        y_true * (-alpha * (torch.ones(size=p.size()) - p) ** gamma * torch.log(p))
        - (torch.ones(size=p.size()) - y_true)
        * (1 - alpha)
        * (p) ** gamma
        * torch.log(p)
    )
    return loss


learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = weighted_focal_loss

In [6]:
# before training
from sklearn.metrics import f1_score
import torch

@torch.no_grad()
def validate():
    m = Softmax(dim=1)
    for i, (batch_x, batch_y) in enumerate(test_loader):
        out = model(batch_x)
        preds = m(out).argmax(dim=1).flatten().numpy()
        ground_truth = batch_y.flatten().numpy()
        acc = (preds == ground_truth).sum() / len(ground_truth)
        print(acc)

        f1 = f1_score(preds, ground_truth, average="weighted")
        print(f1)
        if i >= 5:
            break


validate()

0.6549695073341837
0.6330583274026962
0.4862583705357143
0.4813614819460505
0.49746890943877553
0.5140407216019831
0.5688675860969388
0.5472800879586919
0.6203015385841837
0.6115542827839496
0.5283551897321429
0.525923866370852


In [8]:
from tqdm import tqdm
import torch

device = "cpu"
max_epochs = 1
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = weighted_focal_loss
dataloader = train_loader


# Loop over epochs
loss_list = [0]
scheduler = CosineAnnealingLR(
    optimizer, T_max=len(dataloader.torch_loader) * max_epochs
)
for epoch in range(max_epochs):
    # Training
    loss = 0
    progress_bar = tqdm(total=len(dataloader.torch_loader), desc=f"Loss: {loss:.5f}")

    for i, (local_batch, local_labels) in enumerate(dataloader):
        # Transfer to GPU
        local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        outputs = model(local_batch)
        loss = criterion(outputs, local_labels)  # Calculate the loss
        loss_list.append(loss.item())

        # Backward pass and optimization
        optimizer.zero_grad()  # Clear gradients
        loss.backward()  # Compute gradients
        optimizer.step()  # Update weights
        scheduler.step()  # Update learning rate
        progress_bar.set_description(f"Loss: {loss.item():.5f}")
        progress_bar.update(1)  # Manually update the progress bar

        if i >= 20:
            break

Loss: 0.00000:   0%|          | 0/258 [00:00<?, ?it/s]

Loss: 0.00590:   8%|▊         | 21/258 [01:18<15:23,  3.90s/it]

In [9]:
validate()

0.7200952646683674
0.7841181017732856
0.2683902662627551
0.4083750028728553
0.5371741470025511
0.6722778449771893
0.7111218510841837
0.7858992968100925
0.6607840401785714
0.7515817295525145
0.2718082350127551
0.3969848346868168
